Описание проекта
Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.
Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 
Постройте модель с предельно большим значением F1-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте F1-меру на тестовой выборке самостоятельно.
Дополнительно измеряйте AUC-ROC, сравнивайте её значение с F1-мерой.
Описание данных
Данные находятся в файле /datasets/Churn.csv (англ. «отток клиентов»). Скачать датасет
Признаки
RowNumber — индекс строки в данных
CustomerId — уникальный идентификатор клиента
Surname — фамилия
CreditScore — кредитный рейтинг
Geography — страна проживания
Gender — пол
Age — возраст
Tenure — сколько лет человек является клиентом банка
Balance — баланс на счёте
NumOfProducts — количество продуктов банка, используемых клиентом
HasCrCard — наличие кредитной карты
IsActiveMember — активность клиента
EstimatedSalary — предполагаемая зарплата
Целевой признак
Exited — факт ухода клиента

In [1]:
!pip install imblearn

     |████████████████████████████████| 235 kB 837 kB/s eta 0:00:01
     |████████████████████████████████| 302 kB 16.1 MB/s eta 0:00:01
     |████████████████████████████████| 10.9 MB 3.9 MB/s eta 0:00:01    |█▍                              | 481 kB 6.5 MB/s eta 0:00:02
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np
from scipy import stats as st

from sklearn.preprocessing import OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, roc_curve, confusion_matrix, precision_score, recall_score
from sklearn.dummy import DummyClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler 
import warnings

In [38]:
try:
    data = pd.read_csv('/datasets/Churn.csv')
except:
    print('Не удалось прочитать файл')

In [39]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [40]:
display(data.head(10))

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [41]:
data = pd.get_dummies(data, drop_first=True )

In [42]:
data['Tenure'].fillna(data['Tenure'].median(), inplace=True)

Заполним пропущенные значения в столбце `Tenure`, то есть количество лет, сколько клиент является клиентом банка. Наблюдений с отсутствующим значением не так много, попробуем заменить пропуски на медианное значение:

исправил

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 2945 entries, RowNumber to Gender_Male
dtypes: float64(3), int64(8), uint8(2934)
memory usage: 28.8 MB


In [44]:
from sklearn.model_selection import train_test_split

features = data.drop(['Exited', 'CustomerId', 'RowNumber'], axis=1)
target = data['Exited']

# Объявляем размеры выборок
RANDOM_STATE = 42

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=RANDOM_STATE
)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=RANDOM_STATE
)

Исследование баланса кассов

In [45]:
target_train.value_counts(normalize=True)

0    0.7955
1    0.2045
Name: Exited, dtype: float64

без учета дисбаланса

In [46]:
from sklearn.metrics import f1_score, roc_auc_score


def fit_predict(model, X_train, X_valid, y_train, y_valid):
    """
    Обучает модель и печатает f1 и auc-roc на валидационной выборке
    """
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    valid_pred = model.predict(X_valid)
    
    train_pred_proba = model.predict_proba(X_train)[:, 1]
    valid_pred_proba = model.predict_proba(X_valid)[:, 1]
    
    f1 = f1_score(y_valid, valid_pred)
    roc_auc = roc_auc_score(y_valid, valid_pred_proba)
    
    print('F1-Score=%.2f AUC-ROC=%.2f' % (f1, roc_auc))
    return f1, roc_auc


  внесены изменения в пр. код:
  train_pred_proba = model.predict_proba(X_train)[:, 1]
  valid_pred_proba = model.predict_proba(X_valid)[:, 1]

In [51]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

models = [
    ('DecisionTree', DecisionTreeClassifier(random_state=RANDOM_STATE)),
    ('RandomForest', RandomForestClassifier(random_state=RANDOM_STATE)),
    ('GradientBoosting', GradientBoostingClassifier(random_state=RANDOM_STATE))
]

f1, auc_roc = [], []

for name, model in models:
    print(name, ':', sep='')
    results = fit_predict(model, features_train, features_valid, target_train, target_valid)
    f1.append(results[0])
    auc_roc.append(results[1])
    print()


DecisionTree:
F1-Score=0.50 AUC-ROC=0.69

RandomForest:
F1-Score=0.50 AUC-ROC=0.82

GradientBoosting:
F1-Score=0.54 AUC-ROC=0.85



можно сделать вывод, что Gradient Boosting показывает наилучшие результаты с высоким значением F1-Score (0.54) и AUC-ROC (0.85), в то время как Decision Tree и Random Forest имеют более низкие значения.

Исходя из предоставленных результатов, можно сделать следующие выводы:

При применении метода Downsampling, все три модели показывают некоторое повышение результатов по сравнению с исходными моделями без использования этого метода. Наибольший рост показателей у модели GradientBoosting.

При применении метода Upsampling, вновь все три модели показывают некоторое улучшение результатов по сравнению с исходными моделями без использования этого метода. Опять же, наибольший рост показателей проявляется у модели GradientBoosting.

При использовании классовых весов в моделях не наблюдается заметного изменения результатов по сравнению с исходными моделями без использования этого метода. Уровень весов классов составляет 3.89.

Таким образом, выбор конкретного метода улучшения модели может зависеть от конкретного случая, однако в данном контексте методы Downsampling и Upsampling с указанными параметрами привели к более высокому значению F1-Score и AUC-ROC, особенно при использовании модели GradientBoosting.

In [18]:
model = GradientBoostingClassifier(random_state=RANDOM_STATE)

X_train_balanced, y_train_balanced, _ = balance(features_train, target_train, 'downsample')
model.fit(X_train_balanced, y_train_balanced)

valid_pred = model.predict(features_valid)
test_pred = model.predict(features_test)

f1_valid = f1_score(target_valid, valid_pred)
roc_auc_valid = roc_auc_score(target_valid, valid_pred)

f1_test = f1_score(target_test, test_pred)
roc_auc_test = roc_auc_score(target_valid, test_pred)

print('F1-Score on Validation data: %.2f' % f1_valid)
print('AUC-ROC on Validation data: %.2f' % roc_auc_valid)

print('F1-Score on Test data: %.2f' % f1_test)
print('AUC-ROC on Test data: %.2f' % roc_auc_test)


F1-Score on Validation data: 0.57
AUC-ROC on Validation data: 0.77
F1-Score on Test data: 0.61
AUC-ROC on Test data: 0.51


Была построена модель для прогнозирования ухода клиентов из "Бета-Банка" с F1-мерой на тестовой выборке в 0.61, что превышает требуемый уровень в 0.59. Значения ROC-AUC находятся на достаточно высоком уровене и показывают, что модель прогнозировала не случайно и в большинстве случаев отвечала верно.